In [3]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
import time
import random
from collections import deque # Efficiëntere queue dan een lijst
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [4]:
# --- Configuratie ---
INFINITE_CRAFT_URL = "https://neal.fun/infinite-craft/"
INITIAL_ELEMENTS = ["Water", "Fire", "Wind", "Earth"]
WAIT_TIMEOUT = 0.2  # Seconden wachttijd voor elementen
ACTION_DELAY = 0 # Kleine pauze tussen acties
COMBINATION_DELAY = 0.5 # Wachttijd na een combinatie om resultaat te zien

In [5]:
# --- Hoofdlogica ---
options = uc.ChromeOptions()
#options.add_argument("--headless") # Headless kan problemen geven met detectie/interactie

# Specify your installed Chrome major version (135 in this case)
chrome_major_version = 135

driver = uc.Chrome(options=options, use_subprocess=True, version_main=chrome_major_version)

driver.get(INFINITE_CRAFT_URL)

In [6]:
# //*[@id="sidebar"]/div[3]/div elementen in de sidebar

# //*[@id="instances"] Elements in the game
# //*[@id="instances-top"] element die je vast hebt
# //*[@id="instances instance-hover"] element waar je met over gaat
# //*[@id="#instance instance-pinwheel"] nieuwe element die je hebt gemaakt

In [20]:
elements_sidebar = driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/div/div[1]/div[3]/div/div[1]')
# Maak een lijst van elementen game_stake["elements_sidebar"] in de sidebar elke \n is een nieuwe regel
elements_sidebar_text_list = elements_sidebar[0].text.split("\n")
for i in range(len(elements_sidebar_text_list)):
    # kijk of de eerste letter een letter is haal hem anders weg
    z = 0
    while not elements_sidebar_text_list[i][0].isalpha():
        elements_sidebar_text_list[i] = elements_sidebar_text_list[i][1:]
        z = z + 1
        if z >= len(elements_sidebar_text_list[i]):
            break
    print(f"Element {i}: {elements_sidebar_text_list[i]}") # Print de elementen in de sidebar

Element 0: Water
Element 1: Fire
Element 2: Wind
Element 3: Earth


In [21]:
elements_active = driver.find_elements(By.XPATH, '//*[@id="instances"]/*')

elements_active_text_list = []

for i in range(len(elements_active)):
    active_el = elements_active[i].text
    # haal alles tot en met \n weg
    elements_active_text_list.append(active_el.split("\n")[1])
print("Actieve elementen:", elements_active_text_list)

Actieve elementen: ['Earth']


In [22]:
def get_game_state(driver):
    elements_active = driver.find_elements(By.XPATH, '//*[@id="instances"]/*')
    elements_sidebar = driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/div/div[1]/div[3]/div/div[1]')
    element_hand = driver.find_element(By.XPATH, '//*[@id="instances-top"]')
    try:
        element_hover = driver.find_element(By.XPATH, '//*[@id="instances instance-hover"]')
    except NoSuchElementException:
        element_hover = None
    try:
       element_new = driver.find_element(By.XPATH, '//*[@id="instances instance-pinwheel"]')
    except NoSuchElementException:
        element_new = None

    elements_sidebar_text_list = elements_sidebar[0].text.split("\n")
    for i in range(len(elements_sidebar_text_list)):
        # kijk of de eerste letter een letter is haal hem anders weg
        z = 0
        while not elements_sidebar_text_list[i][0].isalpha():
            elements_sidebar_text_list[i] = elements_sidebar_text_list[i][1:]
            z = z + 1
            if z >= len(elements_sidebar_text_list[i]):
                break
    
    try:
        elements_active_text_list = []
        for i in range(len(elements_active)):
            active_el = elements_active[i].text
            # haal alles tot en met \n weg
            elements_active_text_list.append(active_el.split("\n")[1])

    except StaleElementReferenceException or ElementClickInterceptedException or NoSuchElementException:
        elements_active_text_list = []

    return {"elements_active":elements_active,
            "elements_active_text_list":elements_active_text_list,
            "elements_sidebar":elements_sidebar,
            "elements_sidebar_text_list":elements_sidebar_text_list,
            "element_hand":element_hand,
            "element_hover":element_hover,
            "element_new":element_new}

In [23]:
def get_element_from_sidebar(driver, element_name):
    """Zoekt een specifiek element op naam in de sidebar."""
    # XPath is vaak robuuster voor tekstinhoud, normaliseert spaties
    xpath = f"//div[@class='items-inner']//div[@class='item' and normalize-space(text())='{element_name}']"

    element = WebDriverWait(driver, WAIT_TIMEOUT).until(
        EC.presence_of_element_located((By.XPATH, xpath))
        )
    return element

def get_element_active(driver, element_name):
    """Zoekt alle specifieke elementen op naam in de actieve elementen en retourneert ze als een lijst."""
    # xpath targets the span element containing the text, e.g., <span class="instance-text">Fire</span>
    # within the main game area <div id="instances">
    try:
        xpath = f"//div[@id='instances']//span[@class='instance-text' and text()='{element_name}']"
    
        # Wait for all elements matching the xpath to be present
        elements = WebDriverWait(driver, WAIT_TIMEOUT).until(
            EC.presence_of_all_elements_located((By.XPATH, xpath))
            )
        return elements
    except TimeoutException:
        print(f"Element '{element_name}' niet gevonden in actieve elementen.")
        return []

# print("Zoek naar elementen in de sidebar...")
# element = get_element_from_sidebar(driver, "Wind")
# print("Element gevonden:", element.text)
# print("Zoek naar elementen in de actieve elementen...")
# element = get_element_active(driver, "Wind")
# for elem in element:
#     print("Element gevonden:", elem.text)

In [24]:
def combine_elements(driver,element1_naam, element2_naam, timedelay=0.5):
    """Combineer twee elementen in het spel en retourneer het recept"""
    actions = ActionChains(driver)
    # Zet element 1 in speelbord
    item1_html = get_element_from_sidebar(driver, element1_naam) # Het element dat je wilt combineren
    actions.click(item1_html).perform() # Klik op het eerste element in de sidebar

    #Sla de huidige game state op
    game_state_before = get_game_state(driver)

    #Vind html van element 1
    item1_html_lijst = get_element_active(driver, element1_naam) # Het element dat je wilt combineren
    item1_html_eerstegevonden = item1_html_lijst[0] # Het eerste element dat je wilt combineren

    #Vind element 2 in de sidebar
    item2_html = get_element_from_sidebar(driver, element2_naam)

    #Pak element 2 van de sidebar en sleep het op element 1
    actions.drag_and_drop(item2_html,item1_html_eerstegevonden).perform()
    time.sleep(timedelay) # Wacht op de combinatie om te voltooien

    #Sla game state na op
    game_state_after = get_game_state(driver)

    # Maak een kopie van de lijsten om te bewerken
    before_list1 = game_state_before["elements_active_text_list"]
    before_list1.append(element2_naam)
    after_list1 = game_state_after["elements_active_text_list"]
    before_list2 = before_list1.copy()
    after_list2 =  after_list1.copy()

    for element in before_list1:
        if element in after_list1:
            after_list1.remove(element)

    for element in after_list2:
        if element in before_list2:
            before_list2.remove(element)

    return [before_list2, after_list1]

In [25]:
def clear_canvas(driver):
    """Klikt op de 'Clear' knop om het speelveld leeg te maken."""
        # Zoek de clear knop (selector kan veranderen!)
        # Mogelijke selectors: img[src*='clear.svg'], een div met een specifieke class of id
    clear_button_xpath = "//img[contains(@class, 'clear')]" # Probeer deze XPath
    clear_button = WebDriverWait(driver, WAIT_TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, clear_button_xpath))
    )
    print("Canvas leegmaken...")
        # Soms wordt de klik geblokkeerd, probeer met JavaScript
    driver.execute_script("arguments[0].click();", clear_button)
        # Wacht op de bevestigingsdialog
    time.sleep(ACTION_DELAY)
        # Zoek en klik op de 'Yes' knop (selector kan veranderen!)
        # Mogelijke selectors: button met text 'Yes', specifieke class
    yes_button_xpath = "//button[contains(@class, 'action-btn') and contains(text(), 'Yes')]"
    yes_button = WebDriverWait(driver, WAIT_TIMEOUT).until(
            EC.element_to_be_clickable((By.XPATH, yes_button_xpath))
        )
    driver.execute_script("arguments[0].click();", yes_button)
    print("Canvas leeggemaakt.")
    time.sleep(ACTION_DELAY) # Geef tijd om te verwerken
    return True

In [26]:
def hard_combine_elements(driver,element1_naam, element2_naam, timedelay=0.5, hard_these_elements=False):
    """Combineer twee elementen in het spel en retourneer het recept maar kan ook met zekerheid cobinaties met None vinden"""
    if hard_these_elements:
        clear_canvas(driver)
        recept = combine_elements(driver,element1_naam, element2_naam, timedelay=timedelay)
        if recept == [[], []]:
            print(f"Geen nieuwe elementen gevonden bij het combineren van {element1_naam} en {element2_naam}.")
            return [[element1_naam,element2_naam], [None]]
        else:
            return recept
    else:
        recept = combine_elements(driver,element1_naam, element2_naam, timedelay=timedelay)
        if recept == [[], []]:
            print(f"Geen nieuwe elementen gevonden bij het combineren van {element1_naam} en {element2_naam}.")
            print("Probeer opnieuw... met clear_canvas")
            clear_canvas(driver)
            recept = combine_elements(driver,element1_naam, element2_naam, timedelay=timedelay)
            if recept == [[], []]:
                print(f"Geen nieuwe elementen gevonden bij het combineren van {element1_naam} en {element2_naam}.")
                return [[element1_naam,element2_naam], [None]]
            else:
                return recept
        else:
            print(f"Nieuwe elementen gevonden bij het combineren van {element1_naam} en {element2_naam}: {recept[1]}")
            return recept

In [27]:
actions = ActionChains(driver)
nieuw_element = combine_elements(driver,"Water", "Wind", COMBINATION_DELAY)
print("Nieuw element gemaakt:", nieuw_element)

Nieuw element gemaakt: [['Water', 'Wind'], ['Wave']]


In [28]:
actions = ActionChains(driver)
nieuw_element = hard_combine_elements(driver,"Wind","Wind", COMBINATION_DELAY, hard_these_elements=True)
print("Nieuw element gemaakt:", nieuw_element)

Canvas leegmaken...
Canvas leeggemaakt.
Nieuw element gemaakt: [['Wind', 'Wind'], ['Tornado']]


In [ ]:
recipes = {}  # Dictionary om recepten op te slaan

In [38]:
game_state =  get_game_state(driver)
alle_bekende_elementen = game_state["elements_sidebar_text_list"]

#ik wil een lijst maken van combinaties van elementen die nog niet zijn geprobeerd
combinations = []
for i in range(len(alle_bekende_elementen)):
    for j in range(i, len(alle_bekende_elementen)):
        if (alle_bekende_elementen[i], alle_bekende_elementen[j]) not in recipes:
            combinations.append((alle_bekende_elementen[i], alle_bekende_elementen[j]))

print("Combinaties:", combinations)

Combinaties: [('Water', 'Water'), ('Water', 'Fire'), ('Water', 'Wind'), ('Water', 'Earth'), ('Water', 'Wave'), ('Water', 'Tornado'), ('Water', 'Lake'), ('Water', 'Steam'), ('Water', 'Plant'), ('Water', 'Tsunami'), ('Water', 'Volcano'), ('Water', 'Smoke'), ('Water', 'Lava'), ('Water', 'Dust'), ('Water', 'Storm'), ('Water', 'Mountain'), ('Water', 'Sand'), ('Water', 'Ocean'), ('Water', 'Cloud'), ('Water', 'Swamp'), ('Water', 'Fog'), ('Water', 'Stone'), ('Water', 'Mud'), ('Water', 'Rain'), ('Water', 'Beach'), ('Water', 'Engine'), ('Water', 'Ash'), ('Water', 'Glass'), ('Water', 'Dandelion'), ('Water', 'Hurricane'), ('Water', 'Eruption'), ('Water', 'Sandstorm'), ('Water', 'Avalanche'), ('Water', 'Dune'), ('Water', 'Tree'), ('Water', 'Island'), ('Water', 'Planet'), ('Water', 'Desert'), ('Water', 'Destruction'), ('Water', 'Surf'), ('Water', 'Seaweed'), ('Water', 'Sushi'), ('Water', 'Dust Storm'), ('Water', 'Lily'), ('Water', 'Fjord'), ('Water', 'Tea'), ('Water', 'Titanic'), ('Water', 'Incense'

In [39]:
actions = ActionChains(driver)
for element1, element2 in combinations:
    try:
        nieuw_element = hard_combine_elements(driver,element1, element2, COMBINATION_DELAY)
        # Voeg het nieuwe element toe aan de receptenlijst
        if len(nieuw_element[0]) == 2:  # Ensure nieuw_element[0] has two elements
            recipes[(nieuw_element[0][0], nieuw_element[0][1])] = nieuw_element[1][0]
    except Exception as e:
        print(f"Fout opgetreden bij het combineren van {element1} en {element2}: {e}")
        if "no such window" in str(e):
            print("Webview niet gevonden, sluit de driver en browser.")
            driver.quit()
            break
        continue
    finally:
        # Wacht een korte tijd tussen de acties om overbelasting te voorkomen
        time.sleep(ACTION_DELAY)

Nieuwe elementen gevonden bij het combineren van Water en Water: ['Lake']
Nieuwe elementen gevonden bij het combineren van Water en Fire: ['Steam']
Nieuwe elementen gevonden bij het combineren van Water en Wind: ['Wave']
Nieuwe elementen gevonden bij het combineren van Water en Earth: ['Plant']
Nieuwe elementen gevonden bij het combineren van Water en Wave: ['Tsunami']
Nieuwe elementen gevonden bij het combineren van Water en Tornado: ['Tsunami']
Nieuwe elementen gevonden bij het combineren van Water en Lake: ['Ocean']
Nieuwe elementen gevonden bij het combineren van Water en Steam: ['Cloud']
Nieuwe elementen gevonden bij het combineren van Water en Plant: ['Swamp']
Nieuwe elementen gevonden bij het combineren van Water en Tsunami: ['Ocean']
Nieuwe elementen gevonden bij het combineren van Water en Volcano: ['Lava']
Nieuwe elementen gevonden bij het combineren van Water en Smoke: ['Fog']
Nieuwe elementen gevonden bij het combineren van Water en Lava: ['Stone']
Nieuwe elementen gevonden

KeyboardInterrupt: 

In [40]:
recipes

{('Water', 'Water'): 'Lake',
 ('Water', 'Fire'): 'Steam',
 ('Water', 'Wind'): 'Wave',
 ('Water', 'Earth'): 'Plant',
 ('Water', 'Wave'): 'Tsunami',
 ('Water', 'Tornado'): 'Tsunami',
 ('Water', 'Lake'): 'Ocean',
 ('Water', 'Steam'): 'Cloud',
 ('Water', 'Plant'): 'Swamp',
 ('Water', 'Tsunami'): 'Ocean',
 ('Water', 'Volcano'): 'Lava',
 ('Water', 'Smoke'): 'Fog',
 ('Water', 'Lava'): 'Stone',
 ('Water', 'Dust'): 'Mud',
 ('Water', 'Storm'): 'Rain',
 ('Water', 'Mountain'): 'Lake',
 ('Water', 'Sand'): 'Beach',
 ('Water', 'Ocean'): 'Fish',
 ('Water', 'Cloud'): 'Rain',
 ('Water', 'Swamp'): 'Marsh',
 ('Water', 'Fog'): 'Ice',
 ('Water', 'Stone'): 'Rock',
 ('Water', 'Mud'): 'Swamp',
 ('Water', 'Rain'): 'Rainbow',
 ('Water', 'Beach'): 'Ocean',
 ('Water', 'Engine'): 'Steam',
 ('Water', 'Ash'): 'Puddle',
 ('Water', 'Glass'): 'Wine',
 ('Water', 'Dandelion'): 'Wine',
 ('Water', 'Hurricane'): 'Tsunami',
 ('Fire', 'Fire'): 'Volcano',
 ('Fire', 'Wind'): 'Smoke',
 ('Fire', 'Earth'): 'Lava',
 ('Fire', 'Wave')

# Data opslaan

In [46]:
import csv
import os

# Define the folder name
folder_name = 'recipes'

# Create the folder if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Define the CSV file path
csv_file_path = os.path.join(folder_name, 'discovered_recipes1.csv')

# Write the recipes to the CSV file
try:
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Element1', 'Element2', 'Result']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for (elem1, elem2), result in recipes.items():
            if result == None:
                result = "None"
            writer.writerow({'Element1': elem1, 'Element2': elem2, 'Result': result})
    print(f"Recipes successfully saved to {csv_file_path}")
except Exception as e:
    print(f"Error writing recipes to CSV: {e}")

Recipes successfully saved to recipes\discovered_recipes1.csv


# Tot hier werkt het

In [57]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from collections import defaultdict
import io # For reading string-based CSV data

def load_recipe_data(data_source):
    """
    Loads recipe data from a CSV file path, a CSV string, or a pandas DataFrame.
    Expected columns: 'Element1', 'Element2', 'Result'.
    """
    if isinstance(data_source, str):
        try:
            # Try reading as a file path first
            return pd.read_csv(data_source)
        except FileNotFoundError:
            # If not found, assume it's a CSV string
            return pd.read_csv(io.StringIO(data_source))
        except Exception as e:
            raise ValueError(f"Error reading data_source '{data_source}': {e}")
    elif isinstance(data_source, pd.DataFrame):
        return data_source.copy() # Return a copy to avoid modifying original DF
    else:
        raise ValueError("data_source must be a CSV file path, CSV string, or pandas DataFrame.")

def _calculate_out_degrees(df, all_elements):
    """Helper to calculate out-degrees (ingredient frequency)."""
    out_degree_counts = defaultdict(int)
    for _index, row in df.iterrows():
        out_degree_counts[row['Element1']] += 1
        out_degree_counts[row['Element2']] += 1
    # Ensure all elements have an entry, even if 0
    for el in all_elements:
        _ = out_degree_counts[el] 
    return out_degree_counts

def _caculate_in_degrees(df, all_elements):
    """Helper to calculate in-degrees (result frequency)."""
    in_degree_counts = defaultdict(int)
    for _index, row in df.iterrows():
        in_degree_counts[row['Result']] += 1
    # Ensure all elements have an entry, even if 0
    for el in all_elements:
        _ = in_degree_counts[el] 
    return in_degree_counts

def calculate_min_combination_steps(df_recipes, all_elements_set,
                                    base_elements_list=['Fire', 'Water', 'Earth', 'Wind'],
                                    verbose=False):
    """
    Calculates the minimum number of combination steps to create each element
    from the specified base elements.

    Args:
        df_recipes (pd.DataFrame): DataFrame with 'Element1', 'Element2', 'Result' columns.
        all_elements_set (set): A set of all unique element names in the dataset.
        base_elements_list (list): List of elements considered generation 0.
                                   These elements should not have recipes to create them.
        verbose (bool): If True, prints progress messages.

    Returns:
        dict: A dictionary where keys are element names and values are the
              minimum combination steps (int). Elements unreachable from base
              elements will not be in the dictionary or will have a placeholder
              like float('inf') if explicitly added.
    """
    min_steps = {}

    # 1. Initialize base elements with 0 steps
    for el in base_elements_list:
        if el in all_elements_set:
            min_steps[el] = 0
        elif verbose:
            print(f"Info: Predefined base element '{el}' not found in `all_elements_set`.")

    if not min_steps and verbose and any(el in all_elements_set for el in base_elements_list):
         print("Warning: No valid base elements from the provided list were initialized.")


    # 2. Iteratively determine minimum steps for derived elements
    #    A reasonable upper bound for iterations is the number of unique elements.
    num_total_elements = len(all_elements_set)
    for i in range(num_total_elements + 1): # Max iterations
        made_change_in_iteration = False
        
        for _index, row in df_recipes.iterrows():
            e1, e2, res = row['Element1'], row['Element2'], row['Result']

            # Check if parents' steps are known
            if e1 in min_steps and e2 in min_steps:
                # Steps for this result via this specific recipe
                steps_via_this_recipe = 1 + max(min_steps[e1], min_steps[e2])
                
                current_res_steps = min_steps.get(res) # Get current steps for result, or None

                # If result is new, or this path is shorter, update it
                if current_res_steps is None or steps_via_this_recipe < current_res_steps:
                    # Ensure we are not trying to re-define a base element's steps
                    if res in base_elements_list and min_steps.get(res, -1) == 0:
                        if verbose:
                            print(f"Warning: Recipe '{e1}+{e2}->{res}' attempts to create a base element '{res}'. "
                                  "Base element steps remain 0.")
                        # Skip updating if 'res' is a base element and already has 0 steps.
                        # If it was NOT a base element, or if it was a base element without steps (e.g. typo),
                        # this check allows its first assignment or update.
                        # This assumes base elements truly have no recipes as per the problem.
                        # If they *could* have recipes and still be base, logic might need adjustment.
                    else:
                        min_steps[res] = steps_via_this_recipe
                        made_change_in_iteration = True
        
        if not made_change_in_iteration:
            if verbose:
                print(f"Minimum steps calculation stabilized in {i+1} iterations.")
            break
    else: # For loop completed without break
        if verbose:
            print(f"Warning: Minimum steps calculation reached max iterations ({num_total_elements + 1}). "
                  f"Not all elements might have their final minimum steps if complex paths exist.")

    # 3. (Optional) Identify and mark unreachable elements
    # For elements in all_elements_set but not in min_steps, they are unreachable.
    # You can assign them float('inf') or simply not include them.
    # For this function's current return, they are just not included.
    if verbose:
        unreachable_count = 0
        for el in all_elements_set:
            if el not in min_steps:
                unreachable_count +=1
        if unreachable_count > 0:
            print(f"Info: {unreachable_count} elements were found to be unreachable from the base elements.")
            
    return min_steps

def calculate_element_attributes(df):
    """
    Calculates key attributes: all_elements, out_degree_counts, generation_levels.
    """
    if not {'Element1', 'Element2', 'Result'}.issubset(df.columns):
        raise ValueError("DataFrame must contain 'Element1', 'Element2', and 'Result' columns.")

    all_elements = set(df['Element1']) | set(df['Element2']) | set(df['Result'])
    out_degree_counts = _calculate_out_degrees(df, all_elements)
    generation_levels = calculate_min_combination_steps(df, all_elements)
    in_degree_counts = _caculate_in_degrees(df, all_elements)
    
    return all_elements, out_degree_counts, generation_levels, in_degree_counts

def generate_3d_layout_positions(df_recipes, all_elements, out_degree_counts, generation_levels,in_degree_count,
                                 spring_k=0.1, spring_iterations=70, spring_seed=42,
                                 z_scale_out_degree=4, z_scale_generation=1.0):
    """
    Generates 3D positions for nodes. X,Y from spring_layout, Z is custom.
    """
    G_layout = nx.Graph() # Graph for layout algorithm
    for el in all_elements:
        G_layout.add_node(el)

    # Add temporary edges to guide spring_layout (connects ingredients to results)
    for _index, row in df_recipes.iterrows():
        if row['Element1'] in G_layout and row['Element2'] in G_layout and row['Result'] in G_layout:
            G_layout.add_edge(row['Element1'], row['Result'])
            G_layout.add_edge(row['Element2'], row['Result'])
            # G_layout.add_edge(row['Element1'], row['Element2']) # Optional: connect ingredients

    # Get initial 3D layout (X, Y, Z_spring)
    pos_spring_3d = nx.spring_layout(G_layout, dim=3, k=spring_k, iterations=spring_iterations, seed=spring_seed)

    pos_final_3d = {}
    
    max_out_val = max(out_degree_counts.values()) if out_degree_counts else 1
    max_out_val = 1 if max_out_val == 0 else max_out_val # Avoid division by zero if all out_degrees are 0
    
    max_gen_val = max(generation_levels.values()) if generation_levels else 1
    max_gen_val = 1 if max_gen_val == 0 else max_gen_val # Avoid division by zero if all generations are 0

    max_in_val = max(in_degree_count.values()) if in_degree_count else 1
    max_in_val = 1 if max_in_val == 0 else max_in_val # Avoid division by zero if all in_degrees are 0



    for node in all_elements:
        x_spring, y_spring, _ = pos_spring_3d.get(node, (0,0,0)) # Use spring X, Y

        out_degree = out_degree_counts.get(node, 0)
        gen_level = generation_levels.get(node, max_gen_val) # Default if missing

        out_contrib = (out_degree / max_out_val) * z_scale_out_degree
        gen_contrib = (gen_level / max_gen_val) * z_scale_generation
        in_contrib = (in_degree_count.get(node, 0) / max_in_val) * z_scale_out_degree
        
        custom_z = out_contrib - in_contrib# - gen_contrib
        pos_final_3d[node] = (x_spring, y_spring, custom_z)
        
    return pos_final_3d

def create_plotly_traces(df_recipes, pos_3d, all_elements, out_degree_counts, generation_levels,in_degree_count,
                         node_size_base=5, node_size_multiplier=1.5, colorscale='Viridis'):
    """
    Prepares Plotly Scatter3D traces for nodes and edges.
    """
    node_x, node_y, node_z_plot = [], [], []
    node_text_labels, node_hover_texts, node_sizes, node_colors = [], [], [], []

    max_gen_val = max(generation_levels.values()) if generation_levels else 0

    # Ensure all_elements is a list for consistent iteration order if needed, though set is fine
    # sorted_elements = sorted(list(all_elements))


    for node in all_elements: 
        if node not in pos_3d: 
            # print(f"Warning: Node '{node}' missing from pos_3d dictionary. Skipping.")
            continue

        x, y, z_val = pos_3d[node]
        node_x.append(x)
        node_y.append(y)
        node_z_plot.append(z_val)
        
        node_text_labels.append(node)
        gen = generation_levels.get(node, 'N/A')
        out_deg = out_degree_counts.get(node, 0)
        node_hover_texts.append(f"{node}<br>Gen: {gen}<br>Out-degree: {out_deg}")
        
        node_sizes.append(node_size_base) #TODO fix size nodes + out_deg * node_size_multiplier)
        node_colors.append(generation_levels.get(node, max_gen_val + 1)) # Color by generation

    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z_plot,
        mode='markers+text', text=node_text_labels,
        hovertext=node_hover_texts, hoverinfo='text',
        marker=dict(
            showscale=True, colorscale=colorscale, color=node_colors, size=node_sizes,
            colorbar=dict(thickness=15, title='Generation Level', xanchor='left'),
            line_width=0.5),
        textfont=dict(size=10, color='black'), textposition="top center"
    )

    edge_x, edge_y, edge_z = [], [], []
    for _index, row in df_recipes.iterrows():
        e1, e2, res = row['Element1'], row['Element2'], row['Result']
        
        if not all(elem in pos_3d for elem in [e1, e2, res]):
            # print(f"Warning: Skipping edge for recipe {e1}+{e2}->{res} due to missing node positions for one or more elements.")
            continue

        x0_e1, y0_e1, z0_e1 = pos_3d[e1]
        x1_res, y1_res, z1_res = pos_3d[res]
        edge_x.extend([x0_e1, x1_res, None]); edge_y.extend([y0_e1, y1_res, None]); edge_z.extend([z0_e1, z1_res, None])

        x0_e2, y0_e2, z0_e2 = pos_3d[e2]
        edge_x.extend([x0_e2, x1_res, None]); edge_y.extend([y0_e2, y1_res, None]); edge_z.extend([z0_e2, z1_res, None])

    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        line=dict(width=0.7, color='#888'), hoverinfo='none', mode='lines'
    )
    return node_trace, edge_trace

def plot_recipe_network(node_trace, edge_trace, title="3D Element Combination Network"):
    """
    Creates and shows the Plotly figure. Returns the figure object.
    """
    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title=title, showlegend=False, hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    scene=dict(
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''),
                        zaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title='Height (Out-degree vs Gen)'),
                        aspectmode='data' # 'cube', 'data', 'manual'
                    )))
    fig.show()
    return fig

# --- Main Orchestration Function ---
def visualize_element_recipes(data_source,
                              spring_k=0.5, spring_iterations=70, spring_seed=42,
                              z_scale_out_degree=2.0, z_scale_generation=1.0,
                              node_size_base=5, node_size_multiplier=1.5,
                              plot_title="3D Element Combination Network",
                              colorscale='Viridis',
                              verbose=False):
    """
    Main function to load data, process it, and generate the 3D network visualization.

    Args:
        data_source (str or pd.DataFrame): Path to CSV file, CSV string, or a pandas DataFrame.
                                           Must contain 'Element1', 'Element2', 'Result' columns.
        spring_k (float): k parameter for NetworkX spring_layout.
        spring_iterations (int): Iterations for NetworkX spring_layout.
        spring_seed (int): Random seed for spring_layout.
        z_scale_out_degree (float): Factor scaling out-degree effect on Z-axis (pulls up).
        z_scale_generation (float): Factor scaling generation level effect on Z-axis (pushes down).
        node_size_base (float): Base size for nodes.
        node_size_multiplier (float): Multiplier for out-degree to determine node size increase.
        plot_title (str): Title of the plot.
        colorscale (str): Plotly colorscale name for node colors.
        verbose (bool): If True, print progress messages.

    Returns:
        plotly.graph_objects.Figure: The generated Plotly figure, or None if data is empty.
    """
    if verbose: print("1. Loading data...")
    df_recipes = load_recipe_data(data_source)
    if df_recipes.empty:
        if verbose: print("No data to visualize.")
        return None

    if verbose: print("2. Calculating element attributes...")
    all_elements, out_degree_counts, generation_levels, in_degree_count = calculate_element_attributes(df_recipes)
    
    if verbose:
        print(f"   - Total unique elements: {len(all_elements)}")
        max_gen = max(generation_levels.values()) if generation_levels else 'N/A'
        print(f"   - Max generation level found: {max_gen}")

    if verbose: print("3. Generating 3D layout positions...")
    pos_3d = generate_3d_layout_positions(df_recipes, all_elements, out_degree_counts, generation_levels,in_degree_count,
                                          spring_k, spring_iterations, spring_seed,
                                          z_scale_out_degree, z_scale_generation)

    if verbose: print("4. Creating Plotly traces...")
    node_trace, edge_trace = create_plotly_traces(df_recipes, pos_3d, all_elements,
                                                  out_degree_counts, generation_levels,in_degree_count,
                                                  node_size_base, node_size_multiplier,
                                                  colorscale)

    if verbose: print("5. Generating and showing plot...")
    fig = plot_recipe_network(node_trace, edge_trace, title=plot_title)
    
    return fig


print("--- Running visualization with example data ---")
# Use the embedded CSV string for the example
figure = visualize_element_recipes(
    "recipes\discovered_recipes1.csv",
    spring_k=0.2,          
    spring_iterations=80,
    z_scale_out_degree=0.2, 
    z_scale_generation=1.2, 
    node_size_multiplier=2.0,
    plot_title="Alchemy Recipe Explorer 3D",
    verbose=True # Show progress steps
)

--- Running visualization with example data ---
1. Loading data...
2. Calculating element attributes...
   - Total unique elements: 90
   - Max generation level found: 5
3. Generating 3D layout positions...
4. Creating Plotly traces...
5. Generating and showing plot...
